In [8]:
import os
import sys
import time

from ssd.ssd_vgg16 import create_ssd_vgg16
from ssd.ssd_mobilenetv1 import create_ssd_mobilenetv1
from ssd.ssd_mobilenetv1_lite import create_ssd_mobilenetv1_lite
from ssd.ssd_mobilenetv2_lite import create_ssd_mobilenetv2_lite
from ssd.pre_ssd_mobilenetv1_lite import create_pre_ssd_mobilenetv1_lite
from ssd.pre_ssd_mobilenetv2_lite import create_pre_ssd_mobilenetv2_lite
from ssd.ssd import init_ssd

import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from voc_data import create_batch_generator
from anchor import generate_default_boxes
from losses import create_losses

from tqdm import tqdm
import numpy as np
import os

from anchor import generate_default_boxes
from box_utils import decode, compute_nms
from voc_data import create_batch_generator
from image_utils import ImageVisualizer
from losses import create_losses
from PIL import Image

from settings import *

In [9]:
# 100 epoch당 7-8시간 소요

os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)
os.makedirs(OUTPUT_DETECTS_DIR, exist_ok=True)

visualizer = ImageVisualizer(IDX_TO_NAME, save_dir=OUTPUT_IMAGES_DIR)

In [12]:
ARCH = 'pre_ssd300-mobilenetv2'
CHECKPOINT_DIR = 'checkpoint/pre_mobilenetv2_lite'
CHECKPOINT_PATH = 'checkpoint/pre_mobilenetv2_lite/ssd_epoch_200.h5'

default_boxes = generate_default_boxes(INFO[ARCH])
ssd = create_pre_ssd_mobilenetv2_lite()

In [13]:
pretrained_type = 'specified'
checkpoint_path = CHECKPOINT_PATH

net = init_ssd(ssd, pretrained_type, checkpoint_path)

>>> load_weights


## Load data

In [14]:
batch_generator, info = create_batch_generator(
    DATA_DIR, DATA_YEAR, default_boxes,
    SIZE, batch_size=BATCH_SIZE, num_batches=-1, mode='test')

In [15]:
def predict(confs, locs, default_boxes):

    confs = tf.math.softmax(confs, axis=-1)
    classes = tf.math.argmax(confs, axis=-1)
    scores = tf.math.reduce_max(confs, axis=-1)

    boxes = decode(default_boxes, locs)

    out_boxes = []
    out_labels = []
    out_scores = []

    for c in range(1, NUM_CLASSES):
        cls_scores = confs[:, c]

        score_idx = cls_scores > 0.7

        cls_boxes = boxes[score_idx]
        cls_scores = cls_scores[score_idx]

        nms_idx = compute_nms(cls_boxes, cls_scores, NMS_THRESHOLD, 200)
        cls_boxes = tf.gather(cls_boxes, nms_idx)
        cls_scores = tf.gather(cls_scores, nms_idx)
        cls_labels = [c] * cls_boxes.shape[0]

        out_boxes.append(cls_boxes)
        out_labels.extend(cls_labels)
        out_scores.append(cls_scores)

    out_boxes = tf.concat(out_boxes, axis=0)
    out_scores = tf.concat(out_scores, axis=0)

    boxes = tf.clip_by_value(out_boxes, 0.0, 1.0).numpy()
    classes = np.array(out_labels)
    scores = out_scores.numpy()

    return boxes, classes, scores

## New ver.(Batch)

In [16]:
for i, (arr_filename, arr_tf_imgs, arr_gt_confs, arr_gt_locs) in enumerate(tqdm(batch_generator, total= info['length'] // BATCH_SIZE, 
                                                             desc='Testing...', unit='images')):
    arr_confs, arr_locs = net(arr_tf_imgs)

    # 각 이미지 별
    for confs, locs, filename in zip(arr_confs, arr_locs, arr_filename):
        
        boxes, classes, scores = predict(confs, locs, default_boxes)

        filename = filename.numpy().decode()
        original_image = Image.open(
            os.path.join(info['image_dir'], '{}.jpg'.format(filename)))
        boxes *= original_image.size * 2
        
        log_file = os.path.join('outputs/detects_mobilenetv1', '{}.txt')
        
        for cls, box, score in zip(classes, boxes, scores):
            cls_name = info['idx_to_name'][cls - 1]
            with open(log_file.format(cls_name), 'a') as f:
                f.write('{} {} {} {} {} {}\n'.format(
                    filename,
                    score,
                    *[coord for coord in box]))

Testing...: 612images [13:26,  1.32s/images]                     


## Old ver.(Single)

In [ ]:
def predict(imgs, default_boxes):
    confs, locs = net(imgs)

    confs = tf.squeeze(confs, 0)
    locs = tf.squeeze(locs, 0)

    confs = tf.math.softmax(confs, axis=-1)
    classes = tf.math.argmax(confs, axis=-1)
    scores = tf.math.reduce_max(confs, axis=-1)

    boxes = decode(default_boxes, locs)

    out_boxes = []
    out_labels = []
    out_scores = []

    for c in range(1, NUM_CLASSES):
        cls_scores = confs[:, c]

        score_idx = cls_scores > CLS_SCORE

        cls_boxes = boxes[score_idx]
        cls_scores = cls_scores[score_idx]

        nms_idx = compute_nms(cls_boxes, cls_scores, NMS_THRESHOLD, 200)
        cls_boxes = tf.gather(cls_boxes, nms_idx)
        cls_scores = tf.gather(cls_scores, nms_idx)
        cls_labels = [c] * cls_boxes.shape[0]

        out_boxes.append(cls_boxes)
        out_labels.extend(cls_labels)
        out_scores.append(cls_scores)

    out_boxes = tf.concat(out_boxes, axis=0)
    out_scores = tf.concat(out_scores, axis=0)

    boxes = tf.clip_by_value(out_boxes, 0.0, 1.0).numpy()
    classes = np.array(out_labels)
    scores = out_scores.numpy()

    return boxes, classes, scores


In [ ]:
DATA_DIR = 'dataset'
batch_generator, info = create_batch_generator(
    DATA_DIR, DATA_YEAR, default_boxes,
    SIZE, 1, -1, mode='test')

In [ ]:
for i, (filename, imgs, gt_confs, gt_locs) in enumerate(
    tqdm(batch_generator, total=info['length'],
         desc='Testing...', unit='images')):
    boxes, classes, scores = predict(imgs, default_boxes)
    filename = filename.numpy()[0].decode()
    original_image = Image.open(
        os.path.join(info['image_dir'], '{}.jpg'.format(filename)))
    boxes *= original_image.size * 2
    
#     visualizer.save_image(
#         original_image, boxes, classes, '{}.jpg'.format(filename))

    log_file = os.path.join('outputs/detects', '{}.txt')

    for cls, box, score in zip(classes, boxes, scores):
        cls_name = info['idx_to_name'][cls - 1]
        with open(log_file.format(cls_name), 'a') as f:
            f.write('{} {} {} {} {} {}\n'.format(
                filename,
                score,
                *[coord for coord in box]))